# FiftyOne 脚本

In [7]:
import fiftyone as fo
session = fo.launch_app()

In [8]:
# 获取所有dataset
datasets = fo.list_datasets()
display(datasets)

['jeff_0613-0624_04_ok']

In [9]:
# delete all datasets
for ds_name in datasets:
    fo.delete_dataset(ds_name) 

In [10]:
from pathlib import Path

def fetch_subsequent_dir(data_root: Path, target_subdir_name: Path):
    data_paths = list(data_root.glob(f"*/{target_subdir_name}"))
    # display(data_paths)
    # get sub dir - no target_subdir_name
    subdir_path_list = [data_path.parent for data_path in data_paths]
    # display(subdir_path_list)
    subdir_name_list = [subdir.name for subdir in subdir_path_list]
    # display(subdir_name_list)
    return subdir_path_list, subdir_name_list


# data_root = Path("/home/tianqi/D/01_Projects/01_swd/02_code/pipeline/ultralytics_ty/_ty/01_data/00_test")
data_root = Path("/media/tianqi/16tb/SWD/2024_SWD_data/Middle_result/run_v7")

target_subdir_name = Path("raw_data")
subdir_path_list, subdir_name_list = fetch_subsequent_dir(data_root, target_subdir_name)
display(subdir_path_list)
display(subdir_name_list)

[PosixPath('/media/tianqi/16tb/SWD/2024_SWD_data/Middle_result/run_v7/jeff_0613-0624_04_ok'),
 PosixPath('/media/tianqi/16tb/SWD/2024_SWD_data/Middle_result/run_v7/jeff_0624-0702_01_ok'),
 PosixPath('/media/tianqi/16tb/SWD/2024_SWD_data/Middle_result/run_v7/jeff_0730-0813_01_ok'),
 PosixPath('/media/tianqi/16tb/SWD/2024_SWD_data/Middle_result/run_v7/lloyd_0715-0729_04_ok'),
 PosixPath('/media/tianqi/16tb/SWD/2024_SWD_data/Middle_result/run_v7/ms1_0605-0621_40_ok'),
 PosixPath('/media/tianqi/16tb/SWD/2024_SWD_data/Middle_result/run_v7/ms1_0710-0726_36_ok'),
 PosixPath('/media/tianqi/16tb/SWD/2024_SWD_data/Middle_result/run_v7/ms1_0726-0809_11_ok'),
 PosixPath('/media/tianqi/16tb/SWD/2024_SWD_data/Middle_result/run_v7/ms1_0809-0823_34_ok'),
 PosixPath('/media/tianqi/16tb/SWD/2024_SWD_data/Middle_result/run_v7/ms2_0621-0710_01_ok'),
 PosixPath('/media/tianqi/16tb/SWD/2024_SWD_data/Middle_result/run_v7/ms2_0726-0809_13_ok'),
 PosixPath('/media/tianqi/16tb/SWD/2024_SWD_data/Middle_result/ru

['jeff_0613-0624_04_ok',
 'jeff_0624-0702_01_ok',
 'jeff_0730-0813_01_ok',
 'lloyd_0715-0729_04_ok',
 'ms1_0605-0621_40_ok',
 'ms1_0710-0726_36_ok',
 'ms1_0726-0809_11_ok',
 'ms1_0809-0823_34_ok',
 'ms2_0621-0710_01_ok',
 'ms2_0726-0809_13_ok',
 'ms2_0809-0823_10_ok',
 'sw1_0605-0613_07_ok',
 'sw1_0711-0725_03_ok',
 'sw2_0725-0808_02_ok',
 'sw2_0808-0823_04_ok']

In [32]:
import time
import fiftyone.utils.coco as fouc  
from datetime import date, datetime

for subdir_path, subdir_name in zip(subdir_path_list, subdir_name_list):
    if subdir_name in fo.list_datasets():
        fo.delete_dataset(subdir_name) 
    print(f"subdir_path: {subdir_path}, subdir_name: {subdir_name}")
    dataset = fo.Dataset.from_dir(
        dataset_type=fo.types.COCODetectionDataset,
        name=f"{subdir_name}",
        data_path=subdir_path / "raw_data",
        # labels_path=subdir_path / "output" / "swd_seg_results_coco.json",   
        labels_path=subdir_path / "labels.json",  
        label_field="swd_seg_results_coco",
        label_types="detections",
    )

    # putin rest no annotation image data
    dataset.merge_dir(  
        dataset_dir=subdir_path / "raw_data",  
        dataset_type=fo.types.ImageDirectory,  
        skip_existing=True,  # 跳过已存在的样本  
        insert_new=True,     # 插入新样本  
    )
    
    # fouc.add_coco_labels(
    #     dataset,
    #     "new_annotations",
    #     str(subdir_path / "labels_seg.json"),
    #     {1: "swd"},
    #     label_type="detections",
    #     coco_id_field="filepath",
    # )

    # 添加时间和focus信息,通过file_name获取， 0606_0617_760.jpg 0606_0617表示时间信息 760表示焦点距离
    for sample in dataset:
        file_name = sample.filepath.split("/")[-1]
        time_info = "_".join(file_name.split("_")[:-1])
        focus_info = file_name.split("_")[-1].split(".")[0]
        dt = datetime.strptime(time_info, "%m%d_%H%M")
        dt = dt.replace(year=2024)  # 假设年份为2024年
        sample["time_info"] = dt
        sample["focus_info"] = focus_info
        sample.save()
    
    # 6. 建立索引  
    dataset.create_index("time_info")  
    dataset.create_index("focus_info")
    break

subdir_path: /media/tianqi/16tb/SWD/2024_SWD_data/Middle_result/run_v7/jeff_0613-0624_04_ok, subdir_name: jeff_0613-0624_04_ok
 100% |█████████████████| 226/226 [124.1ms elapsed, 0s remaining, 1.8K samples/s]  
 100% |███████████████| 1112/1112 [109.9ms elapsed, 0s remaining, 10.3K samples/s] 
Converting existing index 'filepath' to unique on dataset '2025.11.17.01.08.14.967608'
Converting existing index 'filepath' to unique on dataset 'jeff_0613-0624_04_ok'
